In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

In [8]:
df=pd.read_csv('diamonds.csv')

# Make test default model

In [9]:
#create log columns
def create_log_columns(df,param):
    cat_price=['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
    df['clarity_log'] = df['clarity'].apply(lambda x: 1+ (cat_price.index(x)))
    color_list = sorted(list(df.color.unique()),reverse=True)
    df['color_log'] = df['color'].apply(lambda x: 1+ (color_list.index(x)))
    cut_price = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
    df['cut_log'] = df['cut'].apply(lambda x: 1+ (cut_price.index(x)))
    if param == 1:
        df['price_log'] = np.log(df['price'])
    df['carat_log']=np.log(df['carat'])
    df['cut_log']=np.log(df['cut_log'])
    df['color_log']=np.log(df['color_log'])
    df['clarity_log']=np.log(df['clarity_log'])
    df['depth_log']=np.log(df['depth'])
    df['table_log']=np.log(df['table'])
    df['x_log']=np.log(df['x']+1)
    df['y_log']=np.log(df['y']+1)
    df['z_log']=np.log(df['z']+1)
    return df

# Make model for categories combination

In [10]:
df = create_log_columns(df,1)
clar_lst = df.clarity.unique()
dfs = [df.query(f'(clarity =="{x}")') for x in clar_lst]

In [11]:
def ger_model(df):
    y= df['price_log']
    X= df[['carat_log','cut_log','color_log','depth_log','table_log','x_log','y_log','z_log']]
    modelo = LinearRegression()
    modelo.fit(X, y)
    y_predict=modelo.predict(X)
    result_model=np.sqrt(metrics.mean_squared_error(y,y_predict))
    result_default=test_default(df)
    return modelo

In [12]:
modelos = [ger_model(df) for df in dfs]

In [13]:
def do_predict(row):
    clar = row['clarity']
    X= [[row['carat_log'],row['cut_log'],row['color_log'],row['depth_log'],row['table_log'],row['x_log'],row['y_log'],row['z_log']]]
    index = list(clar_lst).index(clar)
    model = modelos[index] 
    y_predict = model.predict(X)
    y_predict=np.exp(y_predict)
    return y_predict
 

In [14]:
y_predict = df.apply(lambda row : do_predict(row), axis=1)
y_real = df['price']
metrics.r2_score(y_real, y_predict).round(3)

0.966

In [15]:
np.sqrt(metrics.mean_squared_error(y_real,y_predict))

740.9133966289618

In [16]:
rick_diamonds = pd.read_csv('rick_diamonds.csv')
rick_copy = rick_diamonds.copy()
rick_copy = create_log_columns(rick_copy,0)
y_predict = rick_copy.apply(lambda row : do_predict(row)[0], axis=1)
rick_diamonds['price_predicted'] = y_predict
rick_diamonds.to_csv('./Deliverables/rick_diamonds_mean.csv')